In [ ]:
%load_ext autoreload
%autoreload 2

import polars as pl
import markout

In [ ]:
# load pools
# TODO: can I totally skip this step? Ideally you could build it as an add-on feature to make the analysis extensible.
pools = (
    # see pools.sql
    pl.read_csv('2024.5.2 uniswap pools for markout analysis.csv')
    # I need to filter out for WETH pairs only because I don't have USDC values for everything else
    # I'm filtering in the Python rather than the SQL so that it's extensible to USDC in the future.
    .filter(
        (pl.col('TOKEN0_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
        |
        (pl.col('TOKEN1_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
    )
    .with_columns(
        FEE = (pl.col('FEE')/100).cast(pl.Int8),
        # whether WETH is token0 or not
        # TODO: can I remove this and pass decimals to the markout?
        IS_WETH_TOKEN0 = (
            pl.when(pl.col('TOKEN0_ADDRESS') == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
            .then(True)
            .otherwise(False)
        )
    )
)

In [ ]:
pools.head()

In [ ]:
# FYI BOB/WETH is 0x3887e82dbdbe8ec6db44e6298a2d48af572a3b78
swaps = markout.loadSwaps()

swaps.head(10000).write_csv('swaps_cut.csv')

In [ ]:
# generate the volume per-row

(swaps
 .with_columns(
     volume = pl
        .when(pl.col('token0_symbol') == 'WETH') # if WETH is in token0 slot
        .then(pl.col('amount0')/(10 ** pl.col('token0_decimals'))) # then volume (in WETH) = amount0, decimal-adjusted
        .otherwise(pl.col('amount1')/(10 ** pl.col('token1_decimals'))) # otherwise, WETH is in token1 slot and volume (in WETH = amount1, decimal adjusted)
 )
 .head())

In [ ]:
swaps.head()

In [ ]:
markout_data = (pl.DataFrame(data = {
    # address of the liquidity pool. Assume I have many pools
    'address': ['0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640',
                '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640',
                '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640',
                '0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640', 
                '0xdce93ed9ae7c53143e19cf799d156b72d1cc2777'],
    # decimal-adjusted price of a trade
    'execution_price': [3000,
                        3010,
                        3020,
                        3030,
                        5000000],
    # timestamp of the trade when it occurred
    # you can see that the price is slowly ascending over time
    'timestamp': ['2023-08-07T16:16:59.000000',
                  '2023-08-07T16:20:58.000000', # + 4m 59s
                  '2023-08-07T16:22:58.000000', # + 2m 00s
                  '2023-08-07T16:27:59.000000', # + 5m 01s
                  '2023-08-07T16:27:59.000000'], # partition by address
    'five_min_markout_price_should_be': [3010, # the trade in this row executed at 16:59. Skip to 21:59 then skip backwards to the latest execution price.
                                         3020, # the trade in this row executed at 20:58. Skip to 25:58 then skip backwards to the latest execution price.
                                         3020, # the trade in this row executed at 22:58. Skip to 27:58 then skip backwards to the latest execution price.
                                         3030, # the trade in this row executed at 27:59. Skip to 32:59 then skip backwards to the latest execution price.
                                         5000000] # partition by address
}).with_columns(
    pl.col('timestamp').str.strptime(pl.Datetime)
)
)

markout_data

In [ ]:
# generate the markout price per-row

(markout_data.join_asof(
    other=markout_data.select(
        pl.col('timestamp').dt.offset_by('-5m').alias('timestamp_plus_five_min'),
        pl.col('execution_price'),
        pl.col('address')
    ),
    left_on='timestamp',
    right_on='timestamp_plus_five_min',
    by='address',
    strategy='backward'
).rename({'execution_price_right': 'markout_price'})
)





In [ ]:
# liquidity_pool_address = testing['liquidity_pool_address'].unique().first()

liquidity_pool_address = testing.select(pl.first('liquidity_pool_address'))[0]

filtered_df = testing.filter(pl.col('liquidity_pool_address') == liquidity_pool_address)



In [ ]:
(filtered_df
 .select(['block_timestamp', 'adj_price', 'markout_5m'])
 .write_csv(file='testing.csv')
)

In [ ]:
"""
    TODO:
    something like
    for pool in "pools", which is a pl.DataFrame
    get the pool address and chain
    call load_all_swaps, which returns a pl.DataFrame
    pass the df, TOKEN0_DECIMALS, TOKEN1_DECIMALS, IS_WETH_TOKEN0, and weth_prices to construct_markout
    call execute_markout, which returns a pl.DataFrame
    append the DataFrame to "results", a pl.DataFrame
"""

def process_pools(pools: pl.DataFrame) -> pl.DataFrame:
    # Initialize an empty list to store the results
    results = []

    for pool in pools.iter_rows(named=True):

        pool_address = pool['LIQUIDITY_POOL_ADDRESS']
        chain = 'ethereum' 
        # TODO: lint - these should be lowercase
        TOKEN0_DECIMALS = pool['TOKEN0_DECIMALS']
        TOKEN1_DECIMALS = pool['TOKEN1_DECIMALS']
        IS_WETH_TOKEN0 = pool['IS_WETH_TOKEN0']
        fee = pool['FEE']

        # Load all swaps for the current pool
        swaps_df = markout.load_all_swaps(pool_address, chain)

        # Construct the markout DataFrame
        markout_df = markout.construct_markout(swaps_df, TOKEN0_DECIMALS, TOKEN1_DECIMALS, IS_WETH_TOKEN0)

        # Execute the markout calculation
        markout_result = markout.execute_markout(markout_df)

        # Add pool information to the result
        markout_result['pool_address'] = pool_address
        markout_result['chain'] = chain
        markout_result['fee'] = fee

        # Append the result to the list
        results.append(markout_result)

    # Convert the list of results to a Polars DataFrame
    result_df = pl.from_dicts(results)

    return result_df

# Assuming you have a DataFrame called 'pools' with pool information
# and a DataFrame called 'weth_prices' with WETH price data
result_df = process_pools(pools)

# TODO: then plot. x axis as volume, y axis as markout, can plot fee as dot color.

In [ ]:
bad_results = (result_df.explode('total_volume', 'total_markout')
      .filter(
          (pl.col('total_volume') != 0.0) &
          (pl.col('total_markout') > -508827)
      )
      )

In [ ]:
bad_results.select(["total_volume", "total_markout"]).plot.scatter(x="total_volume", y="total_markout")

In [ ]:
result_df.explode(['total_volume', 'total_markout']).head()
# result_df.explode(['total_volume_usd', 'total_markout_usd']).head().write_csv('results.csv')

In [ ]:
exploded_df = result_df.explode(['total_volume', 'total_markout'])

exploded_df.select(["total_volume", "total_markout"]).plot.scatter(x="total_volume", y="total_markout")